<a href="https://colab.research.google.com/github/NikitaSUAI/EmotionRecognition/blob/main/Hyperparametr_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT

In [ ]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

# !cp -r '/content/drive/MyDrive/OMG_EMO' 'OMG_EMO'

BASE_PATH = Path("/content/drive/MyDrive/OMG_EMO")

TRAIN_PATH = BASE_PATH / "train_set.csv"
VAL_PATH = BASE_PATH / "val_set.csv"
TEST_PATH = BASE_PATH / "test_set.csv"

Mounted at /content/drive


In [ ]:
import pandas as pd

train_df = pd.read_csv(TRAIN_PATH).dropna()
val_df = pd.read_csv(VAL_PATH).dropna()
test_df = pd.read_csv(TEST_PATH).dropna()

# Open feats

In [ ]:
import pickle
with open("/content/drive/MyDrive/OMG_EMO/feats/smile_full_train.pkl", "rb") as f:
  train_opensmile = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/smile_full_val.pkl", "rb") as f:
  val_opensmile = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/smile_full_test.pkl", "rb") as f:
  test_opensmile = pickle.load(f)

# ---

with open("/content/drive/MyDrive/OMG_EMO/feats/fbanks_segment_train.pkl", "rb") as f:
  train_fbanks = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/fbanks_segment_val.pkl", "rb") as f:
  val_fbanks = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/fbanks_segment_test.pkl", "rb") as f:
  test_fbanks = pickle.load(f)

# ---

with open("/content/drive/MyDrive/OMG_EMO/feats/w2v_train.pkl", "rb") as f:
  train_w2v = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/w2v_val.pkl", "rb") as f:
  val_w2v = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/w2v_test.pkl", "rb") as f:
  test_w2v = pickle.load(f)

# ---

with open("/content/drive/MyDrive/OMG_EMO/feats/tf_idf_train.pkl", "rb") as f:
  train_tf_idf = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/tf_idf_val.pkl", "rb") as f:
  val_tf_idf = pickle.load(f)

with open("/content/drive/MyDrive/OMG_EMO/feats/tf_idf_test.pkl", "rb") as f:
  test_tf_idf = pickle.load(f)


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np

def combine_dataset(df, feats_dict, ftype="opensmile"):
  x, clf, val, aro = [], [], [], []
  for idx, row in df.iterrows():
    if ftype == "opensmile":
      x.append(feats_dict[row.path].to_numpy().mean(axis=0)[np.newaxis, :])
    elif ftype == "fbanks":
      x.append(feats_dict[row.path].mean(axis=1)[np.newaxis, :])
    elif ftype == "w2v":
      x.append(feats_dict[row.path].mean(axis=0)[np.newaxis, :])
    elif ftype == "tf_idf":
      # try:
      x.append(feats_dict[row.path][np.newaxis, :])
      # except:
        # ...  
    clf.append(row.EmotionMaxVote)
    val.append(row.valence)
    aro.append(row.arousal)
  
  return {"x" : np.concatenate(x, axis=0), "clf" : np.array(clf),
          "val" : np.array(val), "aro" : np.array(aro)}

train_set  = {
    "opensmile" : combine_dataset(pd.concat([train_df, test_df]),
                                  train_opensmile.update(test_opensmile),
                                  ftype="opensmile"),
    "fbanks" :    combine_dataset(pd.concat([train_df, test_df]),
                                  train_fbanks.update(test_fbanks),
                                  ftype="fbanks"),
    "w2v" :       combine_dataset(pd.concat([train_df, test_df]),
                                  train_w2v.update(test_w2v),
                                  ftype="w2v"),
    "tf_idf" :    combine_dataset(pd.concat([train_df, test_df]),
                                  train_tf_idf.update(test_tf_idf),
                                  ftype="tf_idf"),
    }
test_set   = {
    "opensmile" : combine_dataset(val_df, val_opensmile, ftype="opensmile"),
    "fbanks" :    combine_dataset(val_df, val_fbanks,    ftype="fbanks"),
    "w2v" :       combine_dataset(val_df, val_w2v,       ftype="w2v"),
    "tf_idf" :    combine_dataset(val_df, val_tf_idf,    ftype="tf_idf"),
  }

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np

def combine_dataset(df, feats_dict, ftype="opensmile"):
  x, clf, val, aro = [], [], [], []
  for idx, row in df.iterrows():
    if ftype == "opensmile":
      x.append(feats_dict[row.path].to_numpy().mean(axis=0)[np.newaxis, :])
    elif ftype == "fbanks":
      x.append(feats_dict[row.path].mean(axis=1)[np.newaxis, :])
    elif ftype == "w2v":
      x.append(feats_dict[row.path].mean(axis=0)[np.newaxis, :])
    elif ftype == "tf_idf":
      # try:
      x.append(feats_dict[row.path][np.newaxis, :])
      # except:
        # ...  
    clf.append(row.EmotionMaxVote)
    val.append(row.valence)
    aro.append(row.arousal)
  
  return {"x" : np.concatenate(x, axis=0), "clf" : np.array(clf),
          "val" : np.array(val), "aro" : np.array(aro)}

train_set  = {
    "opensmile" : combine_dataset(train_df, train_opensmile,  ftype="opensmile"),
    "fbanks" :    combine_dataset(train_df, train_fbanks,     ftype="fbanks"),
    "w2v" :       combine_dataset(train_df, train_w2v,        ftype="w2v"),
    # "tf_idf" :    combine_dataset(train_df, train_tf_idf,     ftype="tf_idf"),
    }
test_set   = {
    "opensmile" : combine_dataset(val_df, val_opensmile, ftype="opensmile"),
    "fbanks" :    combine_dataset(val_df, val_fbanks,    ftype="fbanks"),
    "w2v" :       combine_dataset(val_df, val_w2v,       ftype="w2v"),
    # "tf_idf" :    combine_dataset(val_df, val_tf_idf,    ftype="tf_idf"),
  }

# Metrics


In [ ]:
from sklearn.metrics import f1_score
from scipy.stats import pearsonr

def ccc(y_true, y_pred, **kwargs):
    true_mean = np.mean(y_true)
    true_variance = np.var(y_true)
    pred_mean = np.mean(y_pred)
    pred_variance = np.var(y_pred)

    rho,_ = pearsonr(y_pred,y_true)

    std_predictions = np.std(y_pred)

    std_gt = np.std(y_true)


    ccc = 2 * rho * std_gt * std_predictions / (
        std_predictions ** 2 + std_gt ** 2 +
        (pred_mean - true_mean) ** 2)

    return ccc, rho

# Model configs

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, RadiusNeighborsRegressor, RadiusNeighborsClassifier

SVM_PARAMS_OPT = {
    'kernel':('rbf', 'sigmoid', "poly"), 
    "degree":[2, 3], 
    'C':np.linspace(1, 5, 20)}
KNN_PARAMS_OPT = {
        'n_neighbors': [i for i in range(3, 10)],
        'p': [2, 3],  # integer valued parameter
    }
RKNN_PARAMS_OPT = {
        'radius': np.linspace(0.5, 1.5, 20),
        'p': [2, 3],  # integer valued parameter
    }
MODEL_STOR = Path("/content/drive/MyDrive/OMG_EMO/models")

models_discriptions = {
    "SVM_val" : {
        "algo": SVR,
        "param" : SVM_PARAMS_OPT,
        "metrics" : ccc,
        "avg" : None,
        "y" : "val"
    },
    "SVM_aro" : {
        "algo": SVR,
        "param" : SVM_PARAMS_OPT,
        "metrics" : ccc,
        "avg" : None,
        "y" : "aro"
    },
    "SVM" : {
        "algo": SVC,
        "param" : SVM_PARAMS_OPT,
        "metrics" : f1_score,
        "avg" : "micro",
        "y" : "clf"
    },
    "SVM" : {
        "algo": SVC,
        "param" : SVM_PARAMS_OPT,
        "metrics" : f1_score,
        "avg" : "macro",
        "y" : "clf"
    },
    "KNN_val" : {
        "algo": KNeighborsRegressor,
        "param" : KNN_PARAMS_OPT,
        "metrics" : ccc,
        "avg" : None,
        "y" : "val"
    },
    "KNN_aro" : {
        "algo": KNeighborsRegressor,
        "param" : KNN_PARAMS_OPT,
        "metrics" : ccc,
        "avg" : None,
        "y" : "aro"
    },
    "KNN" : {
        "algo": KNeighborsClassifier,
        "param" : KNN_PARAMS_OPT,
        "metrics" : f1_score,
        "avg" : "micro",
        "y" : "clf"
    },
    "KNN" : {
        "algo": KNeighborsClassifier,
        "param" : KNN_PARAMS_OPT,
        "metrics" : f1_score,
        "avg" : "macro",
        "y" : "clf"
    },
    
}

# Pipeline

In [ ]:
from sklearn.model_selection import GridSearchCV

def optim_model(name, discription, train,
                test, iterations=32, cv=3):
  opt = GridSearchCV(
      discription["algo"](),
      discription["param"]
  )
  # opt = discription["algo"]()
  train["x"][np.isnan(train["x"])] = np.take(np.nanmean(train["x"], axis=1),np.isnan(train["x"]))[np.isnan(train["x"])]

  train[discription["y"]][np.isnan(train[discription["y"]])] = np.take(np.nanmean(train[discription["y"]]),np.isnan(train[discription["y"]]))[np.isnan(train[discription["y"]])]

  test["x"][np.isnan(test["x"])] = np.take(np.nanmean(test["x"], axis=1),np.isnan(test["x"]))[np.isnan(test["x"])]

  test[discription["y"]][np.isnan(test[discription["y"]])] = np.take(np.nanmean(test[discription["y"]]),np.isnan(test[discription["y"]]))[np.isnan(test[discription["y"]])]

  opt.fit(train["x"], train[discription["y"]])
  pred = opt.predict(test["x"])
  result_metric = discription["metrics"](test[discription["y"]], pred, average='micro')
  print(f"Target metric : {result_metric}")
  result = {
      "name" : name,
      "metric" : result_metric,
      "type" : discription["y"],
      "param" : opt.cv_results_["params"],
      "rank" : opt.cv_results_["rank_test_score"].tolist(),
      # "param" : "default"
      "predict" : pred.tolist()

  }
  return result

In [ ]:
from tqdm.auto import tqdm

import json
result = []
skipp = []
with tqdm(total=(len(models_discriptions) * len(test_set))) as pbar:
  for key, config in models_discriptions.items():
    for ds_name, dataset in train_set.items():
      if (key, ds_name) in skipp:
        pbar.update(1)
        continue
      pbar.set_description(f"DS: {ds_name}, algo: {key}")
      train = dataset
      test = test_set[ds_name]
      optim_result = optim_model(key, config, train, test, 10)
      optim_result.update({"dataset" : ds_name})
      result.append(optim_result)
      skipp.append((key, ds_name))
      with open("/content/drive/MyDrive/OMG_EMO/results/baseline.json", "w") as f:
        json.dump(result, f)
      with open("/content/drive/MyDrive/OMG_EMO/results/ready.json", "w") as f:
        json.dump(skipp, f)
      pbar.update(1)



  0%|          | 0/18 [00:00<?, ?it/s]

Target metric : (0.05559508202584706, 0.08183624883836277)
Target metric : (-0.025736849128639612, -0.049238033992192946)
Target metric : (0.08992279357698814, 0.12933405313441337)
Target metric : (0.05993475232184466, 0.12930243578911518)
Target metric : (0.16891971894017477, 0.25246145628714584)
Target metric : (0.08027156898000298, 0.15020179843247755)
Target metric : 0.3115727002967359
Target metric : 0.3264094955489614
Target metric : 0.3264094955489614
Target metric : (0.04664010762744643, 0.0713979089599742)
Target metric : (0.039306421101960934, 0.05680193888934035)
Target metric : (0.11042014460261924, 0.15283900173737566)
Target metric : (0.00687891159851412, 0.010374492418432286)
Target metric : (0.21453353303155642, 0.305230278713309)
Target metric : (0.04060596883729332, 0.07129434815435116)
Target metric : 0.2789317507418398
Target metric : 0.3323442136498516
Target metric : 0.3056379821958457


In [ ]:
table = pd.DataFrame(result)
table[table.type == "clf"]

,name,metric,type,param,rank,predict,dataset
6,SVM,0.311573,clf,"[{'C': 1.0, 'degree': 2, 'kernel': 'rbf'}, {'C...","[61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 1...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",opensmile
7,SVM,0.326409,clf,"[{'C': 1.0, 'degree': 2, 'kernel': 'rbf'}, {'C...","[3, 41, 63, 3, 41, 68, 9, 61, 57, 9, 61, 68, 3...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",fbanks
8,SVM,0.326409,clf,"[{'C': 1.0, 'degree': 2, 'kernel': 'rbf'}, {'C...","[1, 81, 4, 1, 81, 6, 12, 83, 3, 12, 83, 8, 8, ...","[4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, ...",w2v
15,KNN,0.278932,clf,"[{'n_neighbors': 3, 'p': 2}, {'n_neighbors': 3...","[13, 14, 11, 12, 10, 9, 8, 6, 7, 5, 4, 2, 1, 3]","[3, 4, 3, 3, 4, 5, 0, 4, 3, 4, 3, 3, 4, 4, 4, ...",opensmile
16,KNN,0.332344,clf,"[{'n_neighbors': 3, 'p': 2}, {'n_neighbors': 3...","[13, 14, 11, 12, 8, 10, 5, 9, 3, 7, 2, 6, 1, 4]","[4, 4, 4, 4, 4, 4, 5, 4, 3, 3, 4, 4, 3, 3, 4, ...",fbanks
17,KNN,0.305638,clf,"[{'n_neighbors': 3, 'p': 2}, {'n_neighbors': 3...","[13, 14, 9, 7, 12, 11, 8, 10, 4, 6, 1, 2, 3, 4]","[4, 3, 0, 3, 3, 4, 5, 3, 3, 4, 4, 4, 4, 3, 3, ...",w2v
